In [2]:
import numpy as np
import toppra.dracula as drac
import toppra as ta
import toppra.algorithm as algo
from toppra import constraint

In [3]:
import re
import ast
import numpy as np

def parse_str_arr(s):
    a = re.sub(r"([^[])\s+([^]])", r"\1, \2", s)
    return np.array(ast.literal_eval(a))

In [14]:
waypts = np.loadtxt('test_waypts_4.txt')
coeff = 1
vlim = coeff * np.vstack([-V_MAX, V_MAX]).T
alim = coeff * np.vstack([-A_MAX, A_MAX]).T

In [10]:
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)

In [11]:
t_sum

0.064046801532567

In [103]:
N_samples = waypts.shape[0]
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)
x = np.linspace(0, 0.15, N_samples)  # magic number
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
pc_vel = constraint.JointVelocityConstraint(vlim)
pc_acc = constraint.JointAccelerationConstraint(
    alim, discretization_scheme=constraint.DiscretizationType.Interpolation
)
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    # gridpoints=gridpoints,
    solver_wrapper="seidel",
)
jnt_traj = instance.compute_trajectory(0, 0)
jnt_traj.cspl.x

 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 82 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 236.208, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.006 secs


array([0.00000000e+00, 3.16199586e-02, 4.60807571e-02, 6.98534902e-02,
       8.96981238e-02, 1.09219610e-01, 1.19978013e-01, 1.31548412e-01,
       1.43994573e-01, 1.57206783e-01, 1.71047139e-01, 1.85690955e-01,
       2.00384167e-01, 2.14353549e-01, 2.40340844e-01, 2.63714702e-01,
       2.86547900e-01, 2.98676786e-01, 3.11448229e-01, 3.24832863e-01,
       3.38772825e-01, 3.52667365e-01, 3.65999132e-01, 3.78726769e-01,
       3.90799480e-01, 4.02279883e-01, 4.13126165e-01, 4.32822977e-01,
       4.49881651e-01, 4.66819611e-01, 4.76227395e-01, 4.85987430e-01,
       4.95860630e-01, 5.05312706e-01, 5.14130230e-01, 5.22532177e-01,
       5.38172225e-01, 5.52096914e-01, 5.78110499e-01, 5.92391505e-01,
       6.05885490e-01, 6.17255324e-01, 6.27890502e-01, 6.38366510e-01,
       6.49605584e-01, 6.62253222e-01, 6.76260358e-01, 6.90817410e-01,
       7.04814861e-01, 7.31597208e-01, 7.45523006e-01, 7.60113422e-01,
       7.74783405e-01, 7.87926466e-01, 8.08726169e-01, 8.41283647e-01,
      

In [113]:
cs = jnt_traj.cspl
x = cs.x
vlim = vlim[:, 1]

In [114]:
vlim

array([1.0875, 1.0875, 1.0875, 1.0875, 1.305 , 1.305 , 1.305 ])

In [119]:
(np.abs(cs.derivative(1)(x))/vlim > 1).any()

False

In [ ]:
data = np.load('/data/toppra/20201204T175646.794406+0000.npz')
waypts = data['waypts']
vlim = data['vlim']
alim = data['alim']
vmax = vlim[:, 1]
amax = alim[:, 1]

In [25]:
waypts = np.array([
[ 0.1088, -1.5201,  1.1266, -2.6961,  1.1851,  1.5092, -1.8498],
[-0.5517, -0.5657,  1.7334, -1.8182,  0.5651,  1.5971, -1.1968],
])
vmax = np.array([0.54375, 0.54375, 0.54375, 0.54375,  0.6525,  0.6525,  0.6525])
amax = np.array([ 3.75, 1.875,   2.5, 3.125,  3.75,     5,     5])
vlim = np.vstack([-vmax, vmax]).T
alim = np.vstack([-amax, amax]).T

In [ ]:
cs = RunTopp(waypts, vlim, alim, return_cs=True, verify_lims=True)

In [95]:
order = 1
lim = vlim*0.893
deriv = cs.derivative(order)(cs.x)
deriv[13, 1] *= -1  # debug
i, j = np.where(~((lim[:, 0] < deriv) & (deriv < lim[:, 1])))
signed_lim = np.where(deriv > 0, lim[:, 1], lim[:, 0])
excess = np.sign(deriv) * (deriv - signed_lim)  # want the +ve entries
excess_percent = excess / np.abs(signed_lim)  # want +ve entries

In [85]:
(np.sign(deriv) * (deriv - signed_lim))[13]

array([-1.49205744e-01,  4.64335721e-04, -1.76552744e-01, -3.84936809e-02,
       -2.66944326e-01, -5.37918975e-01, -2.50138907e-01])

In [83]:
(deriv - signed_lim)[13]

array([ 1.49205744e-01, -4.64335721e-04, -1.76552744e-01, -3.84936809e-02,
        2.66944326e-01, -5.37918975e-01, -2.50138907e-01])

In [64]:
lim

array([[-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.5826825 ,  0.5826825 ],
       [-0.5826825 ,  0.5826825 ],
       [-0.5826825 ,  0.5826825 ]])

In [5]:
vmax - np.abs(cs.derivative(1)(cs.x))[21,:]

array([ 4.67432654e-01, -9.83477545e-05,  4.53298958e-01,  2.05413889e-01,
        6.41458630e-01,  3.03419892e-01,  4.19086688e-01])

In [7]:
cs = RunTopp(waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T175318.489327+0000.npz
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 73 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 1.377, previously 0.217)
 INFO [algorithm.py : 186] Finish parametrization in 0.012 secs
 INFO [__init__.py : 189] Returning optimised cubic spline trajectory of 77 knots


In [56]:
new_waypts = cs(cs.x)
new_waypts.shape

(9, 7)

In [57]:
new_cs = RunTopp(new_waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T160342.125263+0000.npz
WARNING [__init__.py : 85] Duplicates found in input waypoints. This is not recommended, especially for the beginning and the end of the trajectory. Toppra might throw a controllability exception. Attempting to optimise trajectory anyway...
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 5 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 0.095, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.002 secs


### crt jnt test

In [5]:
from toppra.dracula import run_topp_spline, _find_waypts_indices

In [6]:
import numpy as np
data = np.load('/data/toppra/input_data/20210114T160131.877605+0000.npz')
waypts_jnt = data['waypts']
vlim_jnt = data['vlim']
alim_jnt = data['alim']

In [7]:
data = np.load('/data/toppra/input_data/20210114T160131.958179+0000.npz')
waypts_crt = data['waypts']
vlim_crt = data['vlim']
alim_crt = data['alim']

In [ ]:
cs_jnt = run_topp_spline(
    waypts_jnt,
    vlim_jnt,
    alim_jnt,
    verify_lims=True,
    return_cs=True,
)
cs_crt = run_topp_spline(
    waypts_crt,
    vlim_crt,
    alim_crt,
    verify_lims=True,
    return_cs=True,
)

In [ ]:
idx_jnt = _find_waypts_indices(waypts_jnt, cs_jnt)

In [32]:
waypts = waypts_jnt
cs = cs_jnt

idx = np.zeros(waypts.shape[0], dtype=int)
k = 0  # index for knots, scan all knots left to right, start at the 0th
for i, waypt in enumerate(waypts):
    print(f'checking waypt {i}')
    waypt_min_err = float("inf")  # always reset error for current waypt
    while k < cs.x.size:
        print(f'current k: {k}')
        err = np.linalg.norm(cs(cs.x[k]) - waypt)
        if err <= waypt_min_err + (i > 0) * 2e-3:
            print(f'keeping going right, current err {err}')
            waypt_min_err = err
        else:  # we've found the closest knot at the previous knot, k-1
            idx[i] = k - 1
            print(f'found higher err: {err}, diff: {err-waypt_min_err}, set idx[{i}] = {k-1}')
            break
        k += 1
    idx[i] = k - 1
assert idx[0] == 0, "The first knot is not the beginning waypoint"
assert all(
    idx[1:] != 0
), "Failed to find all original waypoints in CubicSpline"
assert idx[-1] == cs.x.size - 1, "The last knot is not the ending waypoint"

checking waypt 0
current k: 0
keeping going right, current err 0.0
current k: 1
found higher err: 2.5132250986948753e-05, diff: 2.5132250986948753e-05, set idx[0] = 0
checking waypt 1
current k: 1
keeping going right, current err 4.612273509455428
current k: 2
keeping going right, current err 4.61230941365929
current k: 3
keeping going right, current err 4.6114082478861596
current k: 4
keeping going right, current err 4.608905454262589
current k: 5
keeping going right, current err 4.6047663878306375
current k: 6
keeping going right, current err 4.599017440087216
current k: 7
keeping going right, current err 4.591685002529233
current k: 8
keeping going right, current err 4.5827954666535975
current k: 9
keeping going right, current err 4.57237522395722
current k: 10
keeping going right, current err 4.560450665937008
current k: 11
keeping going right, current err 4.547048184089872
current k: 12
keeping going right, current err 4.532194169912723
current k: 13
keeping going right, current e

### debug

In [14]:
waypts_str ='''[[-5.13672496e-21  2.17090011e+00  1.97420001e+00 -1.33614886e+00
   1.68947875e+00 -1.43582082e+00 -1.25112307e+00  7.89370894e-01
   1.47137129e+00  1.02121091e+00]
 [-3.32828831e-09  2.17275190e+00  1.97231507e+00 -1.33659756e+00
   1.68829679e+00 -1.43437290e+00 -1.25026381e+00  7.85554409e-01
   1.47030544e+00  1.02154922e+00]
 [-5.00686888e-08  2.19915652e+00  1.94552147e+00 -1.34293866e+00
   1.67151368e+00 -1.41380286e+00 -1.23804998e+00  7.31321335e-01
   1.45512831e+00  1.02636683e+00]
 [-1.51639455e-07  2.25779486e+00  1.88627732e+00 -1.35684395e+00
   1.63446021e+00 -1.36836183e+00 -1.21104920e+00  6.11465275e-01
   1.42149162e+00  1.03704500e+00]
 [-3.03510689e-07  2.34875226e+00  1.79503691e+00 -1.37796271e+00
   1.57753944e+00 -1.29848599e+00 -1.16947937e+00  4.27032262e-01
   1.36948764e+00  1.05355692e+00]
 [-4.78252105e-07  2.46180892e+00  1.68324935e+00 -1.40310061e+00
   1.50815940e+00 -1.21314013e+00 -1.11858237e+00  2.01450363e-01
   1.30527341e+00  1.07395291e+00]
 [-5.95908773e-07  2.55269051e+00  1.59602475e+00 -1.42149901e+00
   1.45461607e+00 -1.14698601e+00 -1.07892489e+00  2.60696374e-02
   1.25434577e+00  1.09014034e+00]
 [-6.31483317e-07  2.61303782e+00  1.54313195e+00 -1.43027127e+00
   1.42328405e+00 -1.10777235e+00 -1.05507600e+00 -7.89981931e-02
   1.22196949e+00  1.10044014e+00]
 [-5.83632527e-07  2.64353275e+00  1.52495861e+00 -1.42886245e+00
   1.41445374e+00 -1.09627032e+00 -1.04788053e+00 -1.12447031e-01
   1.20893061e+00  1.10451031e+00]
 [-4.30989161e-07  2.65223694e+00  1.54129899e+00 -1.41384876e+00
   1.42881119e+00 -1.11507499e+00 -1.06058168e+00 -7.15113729e-02
   1.21707821e+00  1.10131919e+00]
 [-3.04006875e-07  2.66843677e+00  1.55857241e+00 -1.39506161e+00
   1.44321787e+00 -1.13976073e+00 -1.08208919e+00 -2.32865438e-02
   1.23451877e+00  1.09397626e+00]
 [-2.62020137e-07  2.71521425e+00  1.55118251e+00 -1.37953508e+00
   1.44117415e+00 -1.15030921e+00 -1.10065782e+00 -2.01057959e-02
   1.24736989e+00  1.08688331e+00]
 [-2.06513036e-07  2.79397821e+00  1.53620076e+00 -1.35511065e+00
   1.43577600e+00 -1.16541350e+00 -1.13006961e+00 -2.07504742e-02
   1.26745081e+00  1.07552099e+00]
 [-1.14120589e-07  2.88239980e+00  1.52467299e+00 -1.32411373e+00
   1.43375051e+00 -1.18797719e+00 -1.16710234e+00 -8.90599750e-03
   1.29344857e+00  1.06146646e+00]
 [-3.92676220e-08  2.94131780e+00  1.51934671e+00 -1.30186725e+00
   1.43419385e+00 -1.20551252e+00 -1.19357443e+00  4.58445167e-03
   1.31232250e+00  1.05151761e+00]
 [-3.08850145e-09  2.96830320e+00  1.51724136e+00 -1.29145217e+00
   1.43465161e+00 -1.21389806e+00 -1.20595300e+00  1.15570463e-02
   1.32118571e+00  1.04687870e+00]
 [ 5.83989512e-22  2.97048521e+00  1.51709998e+00 -1.29059041e+00
   1.43471062e+00 -1.21460688e+00 -1.20697606e+00  1.21896211e-02
   1.32192147e+00  1.04649639e+00]]'''

vlim_str='''[[-99.       99.     ]
 [ -2.475     2.475  ]
 [ -2.475     2.475  ]
 [ -2.15325   2.15325]
 [ -2.15325   2.15325]
 [ -2.15325   2.15325]
 [ -2.15325   2.15325]
 [ -2.5839    2.5839 ]
 [ -2.5839    2.5839 ]
 [ -2.5839    2.5839 ]] '''

alim_str='''[[-85.     85.   ]
 [ -3.4     3.4  ]
 [ -3.4     3.4  ]
 [-12.75   12.75 ]
 [ -6.375   6.375]
 [ -8.5     8.5  ]
 [-10.625  10.625]
 [-12.75   12.75 ]
 [-17.     17.   ]
 [-17.     17.   ]] '''

In [15]:
waypts = parse_str_arr(waypts_str)
vlim = parse_str_arr(vlim_str)
alim = parse_str_arr(alim_str)

In [65]:
np.savetxt('/src/toppra/tests/dracula/test_waypts_jnt_7.txt', waypts)

In [98]:
import numpy as np
data = np.load('/data/toppra/input_data/20210303T182518.144079+0000.npz')
waypts = data['waypts']
vlim = data['vlim']
alim = data['alim']

In [5]:
from toppra.dracula import run_topp_spline, _check_waypts, run_topp_const_accel

In [ ]:
cs = run_topp_spline(waypts, vlim, alim, return_cs=True)

In [ ]:
cs = run_topp_const_accel(waypts, vlim, alim)

In [20]:
pair_dist = np.diff(waypts, axis=0)  # (N-1, N_dof)
i_sign_flip = (  # find where it changes direction
    np.where(np.sign(pair_dist[:-1]) != np.sign(pair_dist[1:]))[0] + 1
)
pair_t = np.abs(pair_dist) / vlim[:, 1]
pair_t[i_sign_flip] = np.sqrt(  # assume 0 velocity start at max accel
    2 * np.abs(pair_dist)[i_sign_flip] / alim[:, 1]
)
t_sum = pair_t.max(axis=1).sum()
min_pair_dist = np.linalg.norm(pair_dist, axis=1).min()

In [26]:
t_sum

1.3938484764668813

In [61]:
instance.path.duration

0.04181545429400644

In [58]:
A_LIM_EPS = 0.07
V_LIM_EPS = 0.12
pc_acc = constraint.JointAccelerationConstraint(
    alim - np.sign(alim) * A_LIM_EPS,
    discretization_scheme=constraint.DiscretizationType.Interpolation,
)
pc_vel = constraint.JointVelocityConstraint(
            vlim - np.sign(vlim) * V_LIM_EPS
        )
x = np.linspace(0, 0.03*t_sum, waypts.shape[0])
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    solver_wrapper="seidel",
    parametrizer="ParametrizeSpline",
)
traj = instance.compute_trajectory(0, 0)

In [55]:
traj.duration

1.5842264269018105

In [107]:
A_LIM_EPS = 0.07
V_LIM_EPS = 0.12
pc_acc = constraint.JointAccelerationConstraint(
    alim - np.sign(alim) * A_LIM_EPS,
    discretization_scheme=constraint.DiscretizationType.Interpolation,
)
pc_vel = constraint.JointVelocityConstraint(
            vlim - np.sign(vlim) * V_LIM_EPS,
        )
x = np.linspace(0, 0.5*t_sum, waypts.shape[0])
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    solver_wrapper="seidel",
    parametrizer="ParametrizeConstAccel",
    gridpt_min_nb_points=1000,  # ensure eps ~ O(1e-2)
)
traj = instance.compute_trajectory(0, 0)

In [152]:
np.savetxt('/src/toppra/tests/dracula/test_waypts_jnt_6.txt', waypts)

In [154]:
V_MAX = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])
A_MAX = np.array([15, 7.5, 10, 12.5, 15, 20, 20])